In [2]:
! pip install Sastrawi
! pip install tqdm
! pip install indonesian-number-normalizer

In [24]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from indonesian_number_normalizer import create_normalizer
from tqdm import tqdm


In [25]:
nltk.download('punkt')
nltk.download('stopwords')
normalizer = create_normalizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
df = pd.read_csv(r'D:\SKRIPSI\skripsi_2025\fix_dataset\berita_terdahulu.csv')
df

,Title,Content,Sumber,Link,Created_at,label
0,Apakah Eropa Kembali Akan Dilanda Krisis Moneter?,Janji-janji pemilu yang diberikan oleh kubu sa...,Detik,https://news.detik.com/dw/d-7413130/apakah-ero...,"Jumat, 28 Jun 2024 13:49 WIB",positif
1,"Krisis Moneter: Pengertian, Penyebab, dan Damp...",Krisis moneter yang melanda Indonesia pada tah...,Detik,https://finance.detik.com/moneter/d-7090380/kr...,"Kamis, 14 Des 2023 19:45 WIB",negatif
2,"Prabowo Bicara Pertahanan Ekonomi, Ungkit Kris...",Bakal capres Prabowo Subianto menyampaikan pem...,Detik,https://news.detik.com/pemilu/d-7026179/prabow...,"Rabu, 08 Nov 2023 18:01 WIB",positif
3,Sejarah Budi Daya Bunga Kota Batu dan Krisis M...,"Sebelum dikenal sebagai Kota Wisata, masyaraka...",Detik,https://www.detik.com/jatim/wisata/d-7079713/s...,"Jumat, 08 Des 2023 20:08 WIB",positif
4,Memori Taman Puring: Tempat Para Korban PHK Er...,"Terletak di kawasan Kebayoran Baru, Jakarta Se...",Detik,https://finance.detik.com/berita-ekonomi-bisni...,"Jumat, 22 Sep 2023 08:39 WIB",negatif
...,...,...,...,...,...,...
1545,"Resesi Sudah Hampir Pasti, Bakal Separah 1998 ...","Jakarta, CNBC Indonesia - Menteri Keuangan Sri...",CNBC,https://www.cnbcindonesia.com/news/20220927112...,9/27/2022 12:30,negatif
1546,"7 Negara di Ujung Tanduk, Inflasi Tinggi & Ter...","Jakarta, CNBC Indonesia - Krisis global yang t...",CNBC,https://www.cnbcindonesia.com/news/20220913080...,9/13/2022 8:20,negatif
1547,"Dunia di Bibir Jurang Krisis, Lebih Ngeri dari...","Jakarta, CNBC Indonesia - Lonjakan inflasi dan...",CNBC,https://www.cnbcindonesia.com/news/20220621124...,6/21/2022 14:55,negatif
1548,"Jokowi Cemaskan Resesi, Indonesia Bakal Kena?","Jakarta, CNBC Indonesia - Presiden Joko Widodo...",CNBC,https://www.cnbcindonesia.com/news/20220511142...,5/11/2022 16:20,negatif


In [27]:
df = df[['Title', 'Content', 'Created_at', 'label']]

In [28]:
df = df.drop_duplicates(subset=['Content', 'label'])
print(f"Dataset setelah dibersihkan memiliki {len(df)} baris")

Dataset setelah dibersihkan memiliki 1339 baris


In [29]:
df

,Title,Content,Created_at,label
0,Apakah Eropa Kembali Akan Dilanda Krisis Moneter?,Janji-janji pemilu yang diberikan oleh kubu sa...,"Jumat, 28 Jun 2024 13:49 WIB",positif
1,"Krisis Moneter: Pengertian, Penyebab, dan Damp...",Krisis moneter yang melanda Indonesia pada tah...,"Kamis, 14 Des 2023 19:45 WIB",negatif
2,"Prabowo Bicara Pertahanan Ekonomi, Ungkit Kris...",Bakal capres Prabowo Subianto menyampaikan pem...,"Rabu, 08 Nov 2023 18:01 WIB",positif
3,Sejarah Budi Daya Bunga Kota Batu dan Krisis M...,"Sebelum dikenal sebagai Kota Wisata, masyaraka...","Jumat, 08 Des 2023 20:08 WIB",positif
4,Memori Taman Puring: Tempat Para Korban PHK Er...,"Terletak di kawasan Kebayoran Baru, Jakarta Se...","Jumat, 22 Sep 2023 08:39 WIB",negatif
...,...,...,...,...
1536,"Meski Dilanda Demo, Rupiah Menguat 3 Hari vs R...","Jakarta, CNBC Indonesia - Nilai tukar rupiah m...",10/7/2020 19:15,positif
1537,"Cadev RI Anjlok & Demo, Bikin Rupiah KO vs Dol...","Jakarta, CNBC Indonesia - Nilai tukar rupiah m...",10/7/2020 17:12,negatif
1540,"Muncul Tanda-Tanda Ekonomi RI Terancam Gawat, ...","Jakarta, CNBC Indonesia - Kondisi ekonomi RI d...",10/5/2024 21:45,positif
1546,"7 Negara di Ujung Tanduk, Inflasi Tinggi & Ter...","Jakarta, CNBC Indonesia - Krisis global yang t...",9/13/2022 8:20,negatif


PRE PROCESSING DATA

In [30]:
def cleaning(teks):
    if not isinstance(teks, str):
        return ""
    teks = re.sub(r'<.*?>', ' ', teks)
    teks = re.sub(r'https?://\S+|www\.\S+', ' ', teks)
    teks = re.sub(r'ADVERTISEMENT.*?CONTENT', ' ', teks, flags=re.IGNORECASE | re.DOTALL)
    teks = re.sub(r'[^\w\s\d]', ' ', teks)
    teks = re.sub(r'\s+', ' ', teks).strip()
    return teks
df['cleaned_text'] = df['Content'].fillna('').apply(cleaning)

In [31]:
def case_folding(teks):
    return teks.lower()
df['case_folded_text'] = df['cleaned_text'].apply(case_folding)

In [32]:
def normalisasi(teks):
    # Normalisasi angka (misalnya "100 juta" menjadi "seratus juta")
    try:
        # Gunakan normalize_text dari instance normalizer
        teks = normalizer.normalize_text(teks)
    except Exception as e:
        print(f"Error saat normalisasi angka: {e}")
        pass
    slang_dict = {
        'dgn': 'dengan',
        'tdk': 'tidak',
        'tsb': 'tersebut',
        'utk': 'untuk',
        'spy': 'supaya',
        'krn': 'karena',
        'jg': 'juga',
        'bs': 'bisa',
        'sdh': 'sudah',
        'blm': 'belum',
        'org': 'orang',
        'yg': 'yang',
        'sy': 'saya',
        'dlm': 'dalam',
        'pd': 'pada',
        'dr': 'dari',
        'kmrn': 'kemarin',
        'skrg': 'sekarang',
        'hrs': 'harus',
        'msk': 'masuk',
        'trs': 'terus',
        'tp': 'tapi',
        'kalo': 'kalau',
        'gak': 'tidak',
        'ga': 'tidak',
        'ngga': 'tidak',
        'gk': 'tidak',
        'thn': 'tahun',
        'bln': 'bulan',
        'sblm': 'sebelum',
        'stlh': 'setelah',
        'milyar': 'miliar',
        'trilliun': 'triliun',
        'jt': 'juta',
        'rb': 'ribu',
        'persen': 'persen',
        'usd': 'dolar amerika',
        'rupiah': 'rupiah',
        'rp': 'rupiah'
    }
    
    # Buat pattern regex untuk seluruh kamus sekaligus (lebih efisien)
    pattern = r'\b(' + '|'.join(slang_dict.keys()) + r')\b'
    
    # Fungsi replacement yang menggunakan kamus
    def replace_match(match):
        return slang_dict[match.group(0)]
    
    # Ganti semua kata slang dalam satu kali proses
    return re.sub(pattern, replace_match, teks)
df['normalized_text'] = df['case_folded_text'].apply(normalisasi)

In [33]:
def tokenzing(teks):
    return word_tokenize(teks)
df['tokens'] = df['normalized_text'].apply(tokenzing)

In [34]:
indo_stopwords = set(stopwords.words('indonesian'))
tambahan_stopwords = {
    'ya', 'juga', 'dari', 'di', 'ke', 'pada', 'untuk', 'bagi', 'dan', 'atau', 
    'tapi', 'namun', 'dengan', 'secara', 'oleh', 'karena', 'sehingga', 'agar',
    'sebab', 'jika', 'bila', 'adalah', 'ini', 'itu', 'detik', 'kata', 'dalam',
    'saat', 'akan', 'tidak', 'yang', 'belum', 'sudah', 'telah', 'bisa', 'dapat', 
    'nya', 'pak', 'bu', 'hal', 'pun'
}
indo_stopwords.update(tambahan_stopwords)
def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in indo_stopwords]
df['filtered_tokens'] = df['tokens'].apply(remove_stopwords)

In [35]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
def stemming(tokens):
    return [stemmer.stem(word) for word in tokens]
df['stemmed_tokens'] = [stemming(tokens) for tokens in tqdm(df['filtered_tokens'], desc="Stemming")]
df['stemmed_text'] = df['stemmed_tokens'].apply(lambda x: ' '.join(x))

Stemming: 100%|██████████| 1339/1339 [16:17<00:00,  1.37it/s]


In [ ]:
df['processed_content'] = df['stemmed_text']
display(df[['Title', 'processed_content', 'Created_at', 'label']].head())
#df_display = df[['Title', 'processed_content', 'Created_at', 'label']]
#print(df_display.head())

,Title,processed_content,Created_at,label
0,Apakah Eropa Kembali Akan Dilanda Krisis Moneter?,janji janji milu kubu sayap kanan sayap kiri p...,"Jumat, 28 Jun 2024 13:49 WIB",positif
1,"Krisis Moneter: Pengertian, Penyebab, dan Damp...",krisis moneter landa indonesia ribu sembilan r...,"Kamis, 14 Des 2023 19:45 WIB",negatif
2,"Prabowo Bicara Pertahanan Ekonomi, Ungkit Kris...",capres prabowo subianto pikir polemik moneter ...,"Rabu, 08 Nov 2023 18:01 WIB",positif
3,Sejarah Budi Daya Bunga Kota Batu dan Krisis M...,kenal kota wisata masyarakat kota batu tani me...,"Jumat, 08 Des 2023 20:08 WIB",positif
4,Memori Taman Puring: Tempat Para Korban PHK Er...,letak kawasan kebayoran jakarta selatan taman ...,"Jumat, 22 Sep 2023 08:39 WIB",negatif
